In [1]:
import chess.engine
import chess 
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import chess.pgn 
from stockfish import Stockfish
import pandas as pd
import time
import numpy as np
import chess
import chess.pgn
import time
import multiprocessing as mp
import sys
from stockfish import Stockfish
import numpy as np
import json

In [2]:
from maia_functions import *

In [4]:
def extract_features(game):
    # Read the next game
    features = defaultdict(int)
    stockfish=Stockfish("../stockfish/stockfish-windows-x86-64-avx2.exe")
    stockfish.set_depth(15)#How deep the AI looks
    features = defaultdict(int)
    features["white"] = game.headers.get('White', '')
    features["black"] = game.headers.get('Black', '')
    features["moves"] = [str(move) for move in game.mainline_moves()]
    features["WhiteElo"] = game.headers.get('WhiteElo', '')
    features["BlackElo"] = game.headers.get('BlackElo', '')
    
    try:
        mm = maia_cpl(features, stockfish)
        features["maia_cpl_w"] = mm[0]
        features["maia_cpl_b"] = mm[1]
    except Exception as e:
        print(e)
        return features
    return features

In [5]:
pgn_file_path = "../data/landing/fics_openings.pgn"

games = []
pgn_file = open(pgn_file_path)
game = chess.pgn.read_game(pgn_file)

start = 0
limit =40000
count = 0
while game is not None:
    #features = extract_features(game)
    games.append(game)
    
    count += 1
    if count == limit:
        break
    game = chess.pgn.read_game(pgn_file)

pgn_file.close()

In [ ]:
chunk_size = 1000  # Process 1000 games at a time
num_chunks = len(games) // chunk_size + (1 if len(games) % chunk_size > 0 else 0)
for i in range(13, 40):

    start_index = i * chunk_size
    end_index = min(start_index + chunk_size, len(games))
    games_chunk = games[start_index:end_index]

    try:
        result = []
        for game in games_chunk:
            features = extract_features(game)
            result.append(features)

        ndjson_file_path = f'../data/raw/maia/fics_{start_index}.ndjson'
        print(f"saved to {ndjson_file_path}")
        print("--------------------------------------------------------")
        with open(ndjson_file_path, 'w') as f:
            for item in result:
                json.dump(item, f)
                f.write('\n') 
            f.close()
    except Exception as e:
       # By this way we can know about the type of error occurring
        print("The error is: ",e)
    
      